In [71]:
import random
import numpy
import fasttext
from nltk.corpus import stopwords
import csv
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import re

In [72]:
path_csv_file = "latest_hukums_with_classes_csv_file1.csv"

df = pd.read_csv(path_csv_file)
df.head()

,Unnamed: 0.1,Unnamed: 0,ictihat,Suç,YeniSuclar
0,0,0,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",hakaret,Şerefe karşı suçlar
1,1,1,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",tehdit,Hürriyete karşı suçlar
2,2,2,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",yaralama,Vücut dokunulmazlığına karşı suçlar
3,3,3,HÜKÜMLER : Beraat Yerel Mahkemece bozma ...,tehdit,Hürriyete karşı suçlar
4,4,4,HÜKÜMLER : Beraat Yerel Mahkemece bozma ...,hakaret,Şerefe karşı suçlar


In [73]:
label_map = {}
count = 0
for label in df.YeniSuclar.unique():
    label_map[label] = count
    count += 1
df['NUM_LABEL'] = df.YeniSuclar.map(label_map)
#print(label_map)
df.head()

,Unnamed: 0.1,Unnamed: 0,ictihat,Suç,YeniSuclar,NUM_LABEL
0,0,0,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",hakaret,Şerefe karşı suçlar,0
1,1,1,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",tehdit,Hürriyete karşı suçlar,1
2,2,2,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",yaralama,Vücut dokunulmazlığına karşı suçlar,2
3,3,3,HÜKÜMLER : Beraat Yerel Mahkemece bozma ...,tehdit,Hürriyete karşı suçlar,1
4,4,4,HÜKÜMLER : Beraat Yerel Mahkemece bozma ...,hakaret,Şerefe karşı suçlar,0


In [74]:
df.isnull().sum()

Unnamed: 0.1     0
Unnamed: 0       0
ictihat          0
Suç             11
YeniSuclar       0
NUM_LABEL        0
dtype: int64

In [75]:
df = df.dropna(how='any',axis=0)

In [76]:
df.isnull().sum()

Unnamed: 0.1    0
Unnamed: 0      0
ictihat         0
Suç             0
YeniSuclar      0
NUM_LABEL       0
dtype: int64

In [77]:
def clean_text(text):
    alphabetic_only = [word for word in text.split() if word.isalpha()]
    lower_case_only = [word.lower() for word in alphabetic_only]
    stopwords_tr = set(stopwords.words("turkish"))    
    return [word for word in lower_case_only if word not in stopwords_tr]

In [78]:
df.iloc[:, 2] = df.iloc[:, 2].apply(lambda x: " ".join(clean_text(x)))
df = df[['ictihat', 'NUM_LABEL']]
df

,ictihat,NUM_LABEL
0,hükümler hükmün açıklanmasının geri temyiz ist...,0
1,hükümler hükmün açıklanmasının geri temyiz ist...,1
2,hükümler hükmün açıklanmasının geri temyiz ist...,2
3,hükümler beraat yerel mahkemece bozma üzerine ...,1
4,hükümler beraat yerel mahkemece bozma üzerine ...,0
...,...,...
19275,hüküm sanıklar suç işlemek amacıyla örgüt kurm...,14
19276,hüküm bakırköy asliye ceza mahkemesinin esas k...,4
19277,hükümler mahkumiyet yerel mahkemece verilen hü...,0
19278,hükümler mahkumiyet yerel mahkemece verilen hü...,1


In [79]:
df_labeled = df.copy()
df_labeled['NUM_LABEL'] = '__label__' + df_labeled['NUM_LABEL'].astype(str)
df_labeled.head()

,ictihat,NUM_LABEL
0,hükümler hükmün açıklanmasının geri temyiz ist...,__label__0
1,hükümler hükmün açıklanmasının geri temyiz ist...,__label__1
2,hükümler hükmün açıklanmasının geri temyiz ist...,__label__2
3,hükümler beraat yerel mahkemece bozma üzerine ...,__label__1
4,hükümler beraat yerel mahkemece bozma üzerine ...,__label__0


In [80]:
ictihat_label = df_labeled['NUM_LABEL'] + " " + df_labeled['ictihat']
ictihat_label

0        __label__0 hükümler hükmün açıklanmasının geri...
1        __label__1 hükümler hükmün açıklanmasının geri...
2        __label__2 hükümler hükmün açıklanmasının geri...
3        __label__1 hükümler beraat yerel mahkemece boz...
4        __label__0 hükümler beraat yerel mahkemece boz...
                               ...                        
19275    __label__14 hüküm sanıklar suç işlemek amacıyl...
19276    __label__4 hüküm bakırköy asliye ceza mahkemes...
19277    __label__0 hükümler mahkumiyet yerel mahkemece...
19278    __label__1 hükümler mahkumiyet yerel mahkemece...
19279    __label__2 hükümler mahkumiyet yerel mahkemece...
Length: 19269, dtype: object

In [81]:
train, test = train_test_split(ictihat_label, test_size=0.2, random_state=42)

In [82]:
#write test and train into files
f_train = open("train.txt", "w", encoding='utf-8')
for i in range(len(numpy.array(train))):
    f_train.write(numpy.array(train)[i] + "\n")
f_train.close()

f_test = open("test.txt", "w",  encoding='utf-8')
for i in range(len(numpy.array(test))):
    f_test.write(numpy.array(test)[i] + "\n")
f_test.close()

In [108]:
model = fasttext.train_supervised(input="train.txt", epoch=10,  lr=0.5, loss='ova')

In [109]:
model.test("test.txt") #(n, precision, recall)

(3854, 0.6473793461338869, 0.6473793461338869)

In [110]:
model.save_model("model_fasttext.bin")

In [ ]:
#model.predict("any_text")